In [5]:
import os
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import time
import pickle
import theano
from theano import tensor as T
from theano.tensor.nnet import conv2d
from theano.tensor.nnet.nnet import sigmoid, softmax, relu, binary_crossentropy, categorical_crossentropy
from theano.tensor.signal.downsample import max_pool_2d
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStream
from theano.compile.nanguardmode import NanGuardMode
from theano.compile.debugmode import DebugMode
from collections import OrderedDict
from sklearn.metrics import auc, log_loss, precision_recall_curve, roc_auc_score
from prg.prg import create_prg_curve, calc_auprg
from theano import pp
from sklearn.grid_search import ParameterGrid
from sklearn.preprocessing import scale 
np.random.seed(42)
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.core import (
    Activation, Dense, Dropout, Flatten,
    Permute, Reshape, TimeDistributedDense
)
import keras
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.recurrent import GRU
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l1, l2, l1l2
from keras.utils import np_utils
from keras import backend as K
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')


In [4]:
print(plt.style.available)

[u'seaborn-darkgrid', u'seaborn-notebook', u'classic', u'seaborn-ticks', u'grayscale', u'bmh', u'seaborn-talk', u'dark_background', u'ggplot', u'fivethirtyeight', u'seaborn-colorblind', u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', u'seaborn-poster', u'seaborn-muted', u'seaborn-paper', u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', u'seaborn-dark-palette']


In [6]:
import motif_processor
reload(motif_processor)
processorObj = motif_processor.MotifProcessor()
print(len(processorObj.all_motifs))
print(len(processorObj.motif_names))
motif_tensor, motif_names = processorObj.generate_custom_CNN_filters(max_length = 18, padding = 0, truncate = False)

2065
2065


In [52]:
import common_utils
reload(common_utils)

<module 'common_utils' from 'common_utils.py'>

In [9]:
CrossValProcObj = common_utils.CrossValProcessor(output_dirs = ["./HEPG2_act_alt_1_7/", "./HEPG2_rep_alt_1_7/", 
                                                                      "./K562_act_alt_1_7/", "./K562_act_alt_1_7/",
                                                                      "./LCL_act_alt_1_7/"])


# Testing on a single chromosome 

In [10]:
X_train = CrossValProcObj.HepG2_dict['chr1']['X_train'] 
X_test = CrossValProcObj.HepG2_dict['chr1']['X_test'] 
Y_train = CrossValProcObj.HepG2_dict['chr1']['Y_train'] 
Y_test = CrossValProcObj.HepG2_dict['chr1']['Y_test']
chrom_indices = CrossValProcObj.HepG2_dict['chr1']['chrom_indices']
other_indices = CrossValProcObj.HepG2_dict['chr1']['other_indices']
merged_tensor = CrossValProcObj.HepG2_binObj.merged_tensor
merged_tar = CrossValProcObj.HepG2_binObj.merged_tar

In [105]:
import alt_conv_model_1_5_2017
reload(alt_conv_model_1_5_2017)

<module 'alt_conv_model_1_5_2017' from 'alt_conv_model_1_5_2017.py'>

In [106]:
ConvLayerObj = alt_conv_model_1_5_2017.ConvLayer(motif_tensor)

In [107]:
ConvPredObj = alt_conv_model_1_5_2017.ConvPredictor(merged_tensor, merged_tar, motif_tensor.shape, ConvLayerObj)

In [108]:
    CvEngineObj=alt_conv_model_1_5_2017.CvEngine('K562_act', motif_tensor, motif_names, CrossValProcObj.K562_binObj.merged_tensor, CrossValProcObj.K562_binObj.merged_tar, output_dir ="./K562_act_alt_1_7/", debug=False)
    CvEngineObj.start_CV(CrossValProcObj.K562_dict)
    CvEngineObj.summarize(dump_indices = True, dump_weights_bias = True, prefix = 'K562_L2') 
    CvEngineObj=alt_conv_model_1_5_2017.CvEngine('HepG2_act', motif_tensor, motif_names, CrossValProcObj.HepG2_binObj.merged_tensor, CrossValProcObj.HepG2_binObj.merged_tar, output_dir ="./HEPG2_act_alt_1_7/", debug=False)
    CvEngineObj.start_CV(CrossValProcObj.HepG2_dict)
    CvEngineObj.summarize(dump_indices = True, dump_weights_bias = True, prefix = 'HepG2_L2') 
    CvEngineObj=alt_conv_model_1_5_2017.CvEngine('LCL_act', motif_tensor, motif_names, CrossValProcObj.LCL_binObj.merged_tensor, CrossValProcObj.LCL_binObj.merged_tar, output_dir ="./LCL_act_alt_1_7/", debug=False)
    CvEngineObj.start_CV(CrossValProcObj.LCL_dict)
    CvEngineObj.summarize(dump_indices = True, dump_weights_bias = True, prefix = 'LCL_L2')

0.6853	59.0536	0.6150	0.1470	0.3627	0.0000	0.0000	0.0000	143.0000	1134.0000
Completed lin reg on chromsome chr1 in 6.0943 seconds
0.6844	53.1999	0.5535	0.1562	0.3843	0.0000	0.0000	0.0000	82.0000	565.0000
Completed lin reg on chromsome chr10 in 14.3564 seconds
0.6862	51.1178	0.5368	0.1350	-0.1076	0.0000	0.0000	0.0000	84.0000	557.0000
Completed lin reg on chromsome chr11 in 22.0516 seconds
0.6833	57.2177	0.5676	0.1427	0.2510	0.0000	0.0000	0.0000	77.0000	590.0000
Completed lin reg on chromsome chr12 in 29.3351 seconds
0.6791	61.6397	0.6275	0.1451	0.3699	0.0000	0.0000	0.0000	28.0000	274.0000
Completed lin reg on chromsome chr13 in 37.3260 seconds
0.6889	51.6149	0.4824	0.1032	-0.2394	0.0000	0.0000	0.0000	47.0000	361.0000
Completed lin reg on chromsome chr14 in 47.0861 seconds
0.6847	52.8070	0.5728	0.1678	0.3264	0.0000	0.0000	0.0000	57.0000	360.0000
Completed lin reg on chromsome chr15 in 55.5235 seconds
0.6909	53.8473	0.5347	0.2305	0.1909	0.0000	0.0000	0.0000	95.0000	355.0000
Completed lin 

In [94]:
probs, results = ConvPredObj.LogisticRegCVPredict(chrom_indices, other_indices, solver = 'lbfgs', tol=0.0001)

In [109]:
probs, results = ConvPredObj.FC_1layer_model(chrom_indices, other_indices, L1 =0, L2 = 0)

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:1655: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "sequential_5_model" was not an Input tensor, it was generated by layer dropout_11.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: dropout_input_5
  str(x.name))


Epoch 00069: early stopping


In [96]:
BinProcObj = common_utils.ClassificationResult(Y_test, probs)

In [97]:
BinProcObj.self_binary()

In [98]:
str(BinProcObj)

'0.7460\t50.0000\t0.4943\t0.1299\t0.0411\t0.0000\t0.0000\t0.0000\t162.0000\t1057.0000'

In [85]:
ConvPredObj.X_conv[:10]

array([[ 8.3493185 ,  7.95776033,  7.80000067, ...,  7.80000019,
         8.85049152,  8.16976452],
       [ 9.26122475,  7.625     ,  8.01048088, ...,  8.85018826,
         7.81585836,  7.99570751],
       [ 9.0059433 ,  9.14638233,  8.2268734 , ...,  8.65901089,
         8.49999905,  9.0487957 ],
       ..., 
       [ 9.10811424,  8.85716248,  8.1289463 , ...,  8.26907635,
         9.07520962,  7.93333435],
       [ 7.49999905,  7.94916439,  8.090909  , ...,  7.36912346,
         8.60656261,  7.89795017],
       [ 8.32208633,  8.189188  ,  7.61695957, ...,  9.15642262,
         9.31704998,  7.13278675]])